In [7]:
import os
import cv2
import torch
import time
import numpy as np
import subprocess
from datetime import datetime, timedelta
from collections import deque
from ultralytics import YOLO
from shutil import move

# Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SIZE = 640
SEGMENT_DURATION = 600  # 10 minutes in seconds
PROGRESS_INTERVAL = 45  # Seconds between updates
BBOX_PADDING = 0.3
FRAME_SKIP = 5
MIN_SEGMENT_LENGTH = 300  # 5 minutes minimum duration
MIN_CONFIDENCE = 0.4  # Adjusted confidence threshold
HISTORY_BUFFER_SIZE = 900  # 25 second history (assuming 30 fps)

class VideoProcessor:
    def __init__(self):
        self.model = YOLO('../models/bestRefereeDetection.pt').to(DEVICE)
        self.model.fuse()
        if DEVICE == 'cuda':
            self.model.half()
            torch.backends.cudnn.benchmark = True
        self.class_id = self._get_referee_class_id()
        self.detection_history = deque(maxlen=HISTORY_BUFFER_SIZE)
        self.last_valid_bbox = None

    def _get_referee_class_id(self):
        return list(self.model.names.keys())[
            list(self.model.names.values()).index('referee')
        ]

    def process_videos(self, input_dir='../data/input_videos', output_dir='../data/processed_videos', used_dir='../data/used_videos'):
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(used_dir, exist_ok=True)

        for video_file in [f for f in os.listdir(input_dir) if f.endswith('.mp4')]:
            video_path = os.path.join(input_dir, video_file)
            print(f"\n🚀 Starting processing: {video_file}")
            try:
                self._process_single_video(video_path, output_dir, used_dir)
                print(f"\n✅ Successfully processed: {video_file}")
            except Exception as e:
                print(f"\n❌ Error processing {video_file}: {str(e)}")

    def _process_single_video(self, video_path, output_dir, used_dir):
        duration = self._get_video_duration(video_path)
        
        if duration > 3600:
            print("⏳ Splitting into 1-hour chunks...")
            chunks = self._split_into_hourly_chunks(video_path, output_dir)
            print(f"📦 Created {len(chunks)} temporary chunks")
            
            for i, chunk in enumerate(chunks, 1):
                print(f"\n🔧 Processing chunk {i}/{len(chunks)}")
                self._process_video_chunk(chunk, output_dir)
                os.remove(chunk)
                print(f"🧹 Cleaned temporary chunk {i}")
            
            self._safe_move(video_path, used_dir)
            print(f"\n📦 Moved original to: {used_dir}")
            return

        print("🔍 Processing single video chunk...")
        self._process_video_chunk(video_path, output_dir)
        self._safe_move(video_path, used_dir)
        print(f"\n📦 Moved original to: {used_dir}")

    def _process_video_chunk(self, video_path, output_dir):
        cap = cv2.VideoCapture(video_path)
        orig_fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        original_duration = total_frames / orig_fps
        
        base_name = os.path.splitext(os.path.basename(video_path))[0]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        writer = None
        segment_num = 1
        segment_start_frame = 0
        last_progress_update = time.time()
        detections = 0
        frame_counter = 0

        try:
            while frame_counter < total_frames:
                ret, frame = cap.read()
                if not ret:
                    break

                # Process detection every FRAME_SKIP frames
                if frame_counter % FRAME_SKIP == 0:
                    processed_frame, detected = self._process_frame(frame, self.last_valid_bbox)
                    if detected:
                        detections += 1
                        self.last_valid_bbox = self._get_consensus_bbox(frame.shape)
                
                # Use last valid detection for all frames
                if self.last_valid_bbox:
                    x1, y1, x2, y2 = self.last_valid_bbox
                    cropped = frame[y1:y2, x1:x2]
                    processed_frame = cv2.resize(cropped, (MODEL_SIZE, MODEL_SIZE))
                else:
                    processed_frame = np.zeros((MODEL_SIZE, MODEL_SIZE, 3), dtype=np.uint8)

                # Create new segment when needed
                if writer is None or (frame_counter / orig_fps - segment_start_frame / orig_fps >= SEGMENT_DURATION):
                    if writer:
                        writer.release()
                        print(f"💾 Saved segment {segment_num} ({timedelta(seconds=int(SEGMENT_DURATION))})")
                    segment_num += 1
                    writer = self._create_writer(base_name, timestamp, output_dir, segment_num, orig_fps)
                    segment_start_frame = frame_counter
                    print(f"🎬 Started segment {segment_num} at {timedelta(seconds=int(segment_start_frame/orig_fps))}")

                writer.write(processed_frame)
                frame_counter += 1

                # Progress reporting
                if time.time() - last_progress_update >= PROGRESS_INTERVAL:
                    elapsed = frame_counter / orig_fps
                    pct = (elapsed / original_duration) * 100
                    print(f"\n⏱️ Progress [{timedelta(seconds=int(elapsed))}] "
                          f"{pct:.1f}% complete | "
                          f"Segments: {segment_num} | "
                          f"Recent detections: {detections}")
                    last_progress_update = time.time()
                    detections = 0

        finally:
            if writer:
                final_duration = (frame_counter - segment_start_frame) / orig_fps
                if final_duration >= MIN_SEGMENT_LENGTH:
                    writer.release()
                    print(f"💾 Saved final segment {segment_num} ({timedelta(seconds=int(final_duration))})")
                else:
                    writer.release()
                    os.remove(writer.filename)
                    print(f"🧹 Removed short segment (<{MIN_SEGMENT_LENGTH}s)")
            cap.release()

    def _process_frame(self, frame, last_valid):
        original_frame = frame.copy()
        h, w = frame.shape[:2]
        
        # Prepare tensor (BCHW format)
        resized_frame = cv2.resize(frame, (MODEL_SIZE, MODEL_SIZE), interpolation=cv2.INTER_LINEAR)
        tensor = torch.from_numpy(resized_frame).to(DEVICE)
        tensor = tensor.permute(2, 0, 1)
        tensor = tensor.half() if DEVICE == 'cuda' else tensor.float()
        tensor = (tensor / 255.0).unsqueeze(0)
    
        # Run inference with suppressed outputs
        with torch.no_grad():
            results = self.model(tensor, verbose=False)[0]
    
        # Process results with padding
        if len(results.boxes) > 0:
            valid_detections = [box for box in results.boxes.data 
                              if box[4] >= MIN_CONFIDENCE 
                              and (box[2]-box[0])*(box[3]-box[1]) > 1000]
            
            if valid_detections:
                best_detection = max(valid_detections, key=lambda x: x[4])
                self.detection_history.append(best_detection[:4].cpu().numpy())
        
        consensus_bbox = self._get_consensus_bbox(frame.shape)
        
        if consensus_bbox is not None:
            x1, y1, x2, y2 = consensus_bbox
            cropped = original_frame[y1:y2, x1:x2]
            processed_frame = cv2.resize(cropped, (MODEL_SIZE, MODEL_SIZE), interpolation=cv2.INTER_LINEAR)
            return processed_frame, True
        
        return last_valid, False

    def _get_consensus_bbox(self, shape):
        if not self.detection_history:
            return None
        
        # Calculate median of recent detections
        median_bbox = np.median(self.detection_history, axis=0)
        h, w = shape[:2]
        
        # Convert coordinates to original frame scale
        x_scale = w / MODEL_SIZE
        y_scale = h / MODEL_SIZE
        
        # Apply padding and boundary checks
        bbox_width = (median_bbox[2] - median_bbox[0]) * x_scale
        bbox_height = (median_bbox[3] - median_bbox[1]) * y_scale
        pad_x = int(bbox_width * BBOX_PADDING)
        pad_y = int(bbox_height * BBOX_PADDING)
        
        return (
            max(0, int(median_bbox[0] * x_scale - pad_x)),
            max(0, int(median_bbox[1] * y_scale - pad_y)),
            min(w, int(median_bbox[2] * x_scale + pad_x)),
            min(h, int(median_bbox[3] * y_scale + pad_y))
        )


    def _create_writer(self, base_name, timestamp, output_dir, segment_num, fps):
        output_path = os.path.join(
            output_dir,
            f"{base_name}_{timestamp}_part{segment_num:03d}.mp4"
        )
        return cv2.VideoWriter(
            output_path,
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (MODEL_SIZE, MODEL_SIZE)
        )

    def _split_into_hourly_chunks(self, video_path, output_dir):
        base_name = os.path.splitext(os.path.basename(video_path))[0]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_pattern = os.path.join(output_dir, f"{base_name}_{timestamp}_temp_part%03d.mp4")
        
        try:
            subprocess.run([
                'ffmpeg', '-i', video_path,
                '-c', 'copy',
                '-map', '0',
                '-segment_time', '01:00:00',
                '-f', 'segment',
                '-reset_timestamps', '1',
                '-loglevel', 'error',
                output_pattern
            ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError as e:
            print(f"❌ FFmpeg error: {e.stderr.decode()}")
            raise

        return sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) 
                      if f.startswith(f"{base_name}_{timestamp}_temp")])

    def _get_video_duration(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        return frames / fps if fps else 0

    def _safe_move(self, src, dest_dir):
        dest = os.path.join(dest_dir, os.path.basename(src))
        for attempt in range(5):
            try:
                if os.path.exists(dest):
                    os.remove(dest)
                move(src, dest)
                print(f"📤 Moved {os.path.basename(src)} successfully")
                return
            except Exception as e:
                if attempt == 4:
                    print(f"❌ Failed to move {os.path.basename(src)}: {str(e)}")
                time.sleep(2 ** attempt)

if __name__ == "__main__":
    processor = VideoProcessor()
    print("🚀 Starting video processing pipeline...")
    processor.process_videos()
    print("\n🎉 All processing completed successfully!")


YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs
🚀 Starting video processing pipeline...

🚀 Starting processing: videoSplitted.mp4
🔍 Processing single video chunk...
🎬 Started segment 2 at 0:00:00

⏱️ Progress [0:01:25] 6.9% complete | Segments: 2 | Recent detections: 515

⏱️ Progress [0:03:00] 14.5% complete | Segments: 2 | Recent detections: 569
🧹 Removed short segment (<300s)


KeyboardInterrupt: 